===============================================================
# 📘 TP1 - Extracción y Almacenamiento de Datos
## Autor: Pablo Luberriaga
## Fecha: Noviembre 2025
## Fuente:RAWG Video Games Database API
===============================================================

# Objetivo general:
-----------------
Desarrollar un proceso ETL que permita extraer información estática (metadatos) y dinámica (juegos actualizados recientemente)
desde la API pública de RAWG, almacenando los datos en formato Delta Lake para su análisis posterior.

# Objetivos específicos:
----------------------
1. Implementar un proceso de extracción incremental (juegos actualizados recientemente / cambios desde la última ejecución).
2. Implementar una extracción completa o full (metadata).
3. Convertir los datos a DataFrames de Pandas.
4. Guardar los datasets en formato Delta Lake, simulando un entorno de data lake.
5. A futuro preparar la base para análisis posteriores sobre videojuegos (por género,  rating, fecha de lanzamiento, etc.)

# Justificación técnica:
----------------------
- RAWG ofrece endpoints con metadatos (géneros, plataformas, desarrolladores) y listados de juegos que varían en el tiempo (ranking, novedades).


# Enpoints Utilizados


**URL Base:** `https://api.rawg.io/api`

---

##  Ingesta Full - Metadatos

Estos endpoints se utilizan para obtener información estática o de referencia sobre juegos, desarrolladores, etc.

| Endpoint        | Descripción                                           |
|-----------------|-------------------------------------------------------|
| `/genres`       | Lista de géneros de videojuegos.                     |
| `/developers`   | Información sobre desarrolladores de juegos.        |
| `/publishers`   | Información sobre editoras de videojuegos.          |
| `/stores`       | Información sobre tiendas de videojuegos.           |
| `/tags`         | Etiquetas asociadas a juegos.                        |
| `/creators`     | Información sobre creadores de contenido o juegos.  |



##  Ingesta Incremental  -  Videojuegos

Estos endpoints se utilizan para obtener información estática o de referencia sobre juegos, desarrolladores,  etc.

| Endpoint        | Descripción                                           |
|-----------------|-------------------------------------------------------|
| `/games`        | Lista de videojuegos publicados                       |

# 1. IMPORTACIÓN DE LIBRERÍAS

In [1]:
# Transformación y manejo de datos
import pandas as pd
from datetime import datetime, timedelta,timezone
import numpy as np 
import ast
import json
# Sistema de archivos y rutas
from pathlib import Path


# Funciones
from utils import get_data,almacenamiento_datalake_merge,castear_json,limpiar_json,procesar_dimension,agregar_faltantes,almacenamiento_datalake_overwrite

# Configuración
from configparser import ConfigParser

# Almacenamiento en Delta Lake
from deltalake import DeltaTable

# 2. CONFIGURACIÓN Y CREDENCIALES

In [2]:
# Lectura de archivo de configuración
parser = ConfigParser()
parser.read("pipeline.conf")

['pipeline.conf']

In [3]:
# Configuraciones para MinIO
s3_config=parser['S3_STORAGE']
storage_options = {
    'AWS_ENDPOINT_URL': s3_config['AWS_ENDPOINT_URL'],
    'AWS_ACCESS_KEY_ID': s3_config['AWS_ACCESS_KEY_ID'], 
    'AWS_SECRET_ACCESS_KEY': s3_config['AWS_SECRET_ACCESS_KEY'], 
    'AWS_ALLOW_HTTP': s3_config['AWS_ALLOW_HTTP'],
    'aws_conditional_put': s3_config['AWS_CONDITIONAL_PUT'],
    'AWS_S3_ALLOW_UNSAFE_RENAME': s3_config['AWS_S3_ALLOW_UNSAFE_RENAME']
}

# Nombre del bucket
bkt_name = "pabloluberriaga-bucket"

In [4]:
# --------------------------------
#  Configuración Creedenciales y URL BASE
# --------------------------------
# Url base
BASE_URL = "https://api.rawg.io/api"

# Creedencial
api_credentials = parser["RAWG"]


# 3. EXTRACCIÓN DE DATOS - CAPA BRONZE


## 3.1 Ingesta Full - Metadatos

### Extracción

In [5]:
# -------------------------------------
#  Configuración de parámetros de la API
# -------------------------------------
params = {
    "key": api_credentials["API_KEY"],      
    "page_size": api_credentials.get("PAGE_SIZE", 100)
}

# Endpoints de metadatos
endpoints = ["genres", "developers", "publishers", "stores", "tags", "creators","platforms"]
# Diccionario donde guardaremos los metadatos
metadata = {}

# -------------------------------------
#  Extracción de metadatos
# -------------------------------------
for endpoint in endpoints:
    try:
        data = get_data(BASE_URL, endpoint, data_field="results", params=params)
        metadata[endpoint] = data or []
        print(f" {endpoint}: {len(metadata[endpoint])} registros obtenidos")
    except Exception as e:
        print(f" Error extrayendo {endpoint}: {e}")
        metadata[endpoint] = []

 genres: 19 registros obtenidos
 developers: 40 registros obtenidos
 publishers: 40 registros obtenidos
La petición ha fallado. Código de error : HTTPSConnectionPool(host='api.rawg.io', port=443): Max retries exceeded with url: /api/stores?key=13e37963bbd34bf49677bbce97460c2f&page_size=100 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)')))
 stores: 0 registros obtenidos
La petición ha fallado. Código de error : HTTPSConnectionPool(host='api.rawg.io', port=443): Max retries exceeded with url: /api/tags?key=13e37963bbd34bf49677bbce97460c2f&page_size=100 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)')))
 tags: 0 registros obtenidos
La petición ha fallado. Código de error : HTTPSConnectionPool(host='api.rawg.io', port=443): Max retries exceeded with url: /api/creators?key=13e37963bbd34bf49677bbce97460c2f&page_size=100 (Caused by SSLError

In [6]:
# -------------------------------------
#  Conversión a DataFrames
# -------------------------------------
dfs = {}
for key, items in metadata.items():
    print(f" Creando DataFrame para: {key}")
    dfs[key] = pd.json_normalize(items) if items else pd.DataFrame()

# Hago una limpeza previa con platforms por un problema con los datos anidados en el guardado    
dfs['platforms'] = dfs['platforms'].drop(
    columns=['games', 'slug', 'image', 'year_end', 'year_start'],errors='ignore'
)

 Creando DataFrame para: genres
 Creando DataFrame para: developers
 Creando DataFrame para: publishers
 Creando DataFrame para: stores
 Creando DataFrame para: tags
 Creando DataFrame para: creators
 Creando DataFrame para: platforms


Si bien son metadatos, pueden llegar a tener cambios a futuro agregaciones o modificaciones asi que  utilizare tambien un merge para guardarlos en el Data Lakehouse

### ALMACENAMIENTO - CAPA BRONZE - METADATOS 

In [7]:
# -------------------------------------
#  Almacenamiento en formato Delta Lake
# -------------------------------------
for key, df in dfs.items():
    if df.empty:
        print(f" No hay datos para {key}, se omite el guardado.")
        continue

    path_metadata = f"s3://{bkt_name}/bronze/Rawg_Games_API/metadata/{key}"
    

    try:
        almacenamiento_datalake_merge(df, path_metadata, storage_options=storage_options)
    except Exception as e:
        print(f" Error guardando {key} en Delta Lake: {e}")



📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 19 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 40 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 40 registros insertados, 0 registros actualizados.
 No hay datos para stores, se omite el guardado.
 No hay datos para tags, se omite el guardado.
 No hay datos para creators, se omite el guardado.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 40 registros insertados, 0 registros actualizados.


In [8]:
# -------------------------------------
# Verificación de los datos guardados
# -------------------------------------
for key in dfs.keys():    
    path_metadata = f"s3://{bkt_name}/bronze/Rawg_Games_API/metadata/{key}"

    print(f"\n📂 Contenido de la tabla guardada para '{key}':  ({path_metadata})")

    try:
        metadata_df = DeltaTable(path_metadata, storage_options=storage_options).to_pandas()
        display(metadata_df.head(5))
    except Exception as e:
        print(f"⚠️ No se pudo leer {key}: {e}")


📂 Contenido de la tabla guardada para 'genres':  (s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/genres)


,id,name,slug,games_count,image_background,games
0,4,Action,action,191313,https://media.rawg.io/media/games/d82/d82990b9...,"[{'added': 22341, 'id': 3498, 'name': 'Grand T..."
1,51,Indie,indie,86115,https://media.rawg.io/media/games/d1f/d1f872a4...,"[{'added': 14084, 'id': 1030, 'name': 'Limbo',..."
2,3,Adventure,adventure,151509,https://media.rawg.io/media/games/d5a/d5a24f9f...,"[{'added': 15852, 'id': 3439, 'name': 'Life is..."
3,5,RPG,role-playing-games-rpg,61851,https://media.rawg.io/media/games/5a4/5a441122...,"[{'added': 22013, 'id': 3328, 'name': 'The Wit..."
4,10,Strategy,strategy,62318,https://media.rawg.io/media/games/238/2383a172...,"[{'added': 9871, 'id': 10243, 'name': 'Company..."



📂 Contenido de la tabla guardada para 'developers':  (s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/developers)


,id,name,slug,games_count,image_background,games
0,1612,Valve Software,valve-software,44,https://media.rawg.io/media/games/46d/46d98e69...,"[{'added': 20761, 'id': 4200, 'name': 'Portal ..."
1,405,Ubisoft,ubisoft,349,https://media.rawg.io/media/games/15c/15c95a49...,"[{'added': 10728, 'id': 4161, 'name': 'Far Cry..."
2,18893,Feral Interactive,feral-interactive,107,https://media.rawg.io/media/screenshots/375/37...,"[{'added': 12333, 'id': 7689, 'name': 'Rise of..."
3,3709,Ubisoft Montreal,ubisoft-montreal,121,https://media.rawg.io/media/games/99b/99b39612...,"[{'added': 10728, 'id': 4161, 'name': 'Far Cry..."
4,4132,Square Enix,square-enix,372,https://media.rawg.io/media/games/968/968e6210...,"[{'added': 10858, 'id': 10035, 'name': 'Hitman..."



📂 Contenido de la tabla guardada para 'publishers':  (s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/publishers)


...

,id,name,slug,games_count,image_background,language,games
0,31,Singleplayer,singleplayer,250596,https://media.rawg.io/media/games/4be/4be6a6ad...,eng,"[{'added': 22341, 'id': 3498, 'name': 'Grand T..."
1,40847,Steam Achievements,steam-achievements,51616,https://media.rawg.io/media/games/b45/b45575f3...,eng,"[{'added': 22341, 'id': 3498, 'name': 'Grand T..."
2,7,Multiplayer,multiplayer,42503,https://media.rawg.io/media/games/511/5118aff5...,eng,"[{'added': 22341, 'id': 3498, 'name': 'Grand T..."
3,40836,Full controller support,full-controller-support,24099,https://media.rawg.io/media/games/49c/49c3dfa4...,eng,"[{'added': 22341, 'id': 3498, 'name': 'Grand T..."
4,40849,Steam Cloud,steam-cloud,25673,https://media.rawg.io/media/games/198/1988a337...,eng,"[{'added': 20761, 'id': 4200, 'name': 'Portal ..."



📂 Contenido de la tabla guardada para 'creators':  (s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/creators)


,id,name,slug,image,image_background,games_count,positions,games
0,31,Gabe Newell,gabe-newell,https://media.rawg.io/media/persons/5e5/5e5e06...,https://media.rawg.io/media/games/2ba/2bac0e87...,24,"[{'id': 2, 'name': 'director', 'slug': 'direct...","[{'added': 20761, 'id': 4200, 'name': 'Portal ..."
1,37,Marc Laidlaw,marc-laidlaw,https://media.rawg.io/media/persons/da3/da3fc9...,https://media.rawg.io/media/games/2ba/2bac0e87...,15,"[{'id': 1, 'name': 'writer', 'slug': 'writer'}]","[{'added': 20761, 'id': 4200, 'name': 'Portal ..."
2,63,Robin Walker,robin-walker,https://media.rawg.io/media/persons/b76/b76df2...,https://media.rawg.io/media/games/2ba/2bac0e87...,14,"[{'id': 6, 'name': 'designer', 'slug': 'design...","[{'added': 20761, 'id': 4200, 'name': 'Portal ..."
3,27630,David Speyrer,david-speyrer,https://media.rawg.io/media/persons/513/51388a...,https://media.rawg.io/media/games/2ba/2bac0e87...,12,"[{'id': 7, 'name': 'programmer', 'slug': 'prog...","[{'added': 20761, 'id': 4200, 'name': 'Portal ..."
4,72,Jesper Kyd,jesper-kyd,https://media.rawg.io/media/persons/2a5/2a502a...,https://media.rawg.io/media/games/49c/49c3dfa4...,49,"[{'id': 3, 'name': 'composer', 'slug': 'compos...","[{'added': 15951, 'id': 802, 'name': 'Borderla..."



📂 Contenido de la tabla guardada para 'platforms':  (s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/platforms)


,id,name,games_count,image_background
0,4,PC,560064,https://media.rawg.io/media/games/021/021c4e21...
1,187,PlayStation 5,1379,https://media.rawg.io/media/games/618/618c2031...
2,1,Xbox One,5720,https://media.rawg.io/media/games/562/56255381...
3,18,PlayStation 4,6959,https://media.rawg.io/media/games/4cf/4cfc6b7f...
4,186,Xbox Series S/X,1189,https://media.rawg.io/media/games/e44/e445335e...


## 3.2  Ingesta incremental

In [9]:
# --------------------------------
#  Configuración base
# --------------------------------
path_games = f"s3://{bkt_name}/bronze/Rawg_Games_API/games"
hours_back = 24  # cantidad de horas atrás


# -----------------------------
# Determinar la última actualización
# -----------------------------
if DeltaTable.is_deltatable(path_games, storage_options=storage_options):
    df_actual = DeltaTable(path_games, storage_options=storage_options).to_pandas()
    if not df_actual.empty:
        print("📂 Tabla existente. Revisando última actualización...")
        df_actual["updated"] = pd.to_datetime(df_actual["updated"], utc=True)
        # Tomamos la última actualización +1 minuto para no duplicar
        last_updated = df_actual["updated"].max() + pd.Timedelta(minutes=1)
    else:
        print("📂 Tabla vacía. Traemos últimos 24 horas.")
        last_updated = datetime.now(timezone.utc) - timedelta(hours=hours_back)
else:
    print("📂 Tabla no existe. Traemos últimos 24 horas.")
    last_updated = datetime.now(timezone.utc) - timedelta(hours=hours_back)

# Formato requerido por la API
last_updated_str = last_updated.strftime("%Y-%m-%dT%H:%M:%SZ")
today_str = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# -----------------------------
# Parámetros de la API
# -----------------------------
endpoint_dinamico = "games"
params_games = {
    "key": api_credentials["API_KEY"],
    "updated": f"{last_updated_str},{today_str}",
    "ordering": "-updated",
    "page_size": 40
}

# -----------------------------
# Extracción con paginación
# -----------------------------
page = 1
games_data = []

print(f"📅 Extrayendo juegos actualizados entre {last_updated_str} y {today_str}")
while True:
    params_games["page"] = page
    data = get_data(BASE_URL, endpoint_dinamico, data_field="results", params=params_games)
    if not data:
        break
    games_data.extend(data)
    if len(data) < params_games["page_size"]:
        break
    page += 1

# -----------------------------
# Transformo en DataFrame
# -----------------------------
df_games = pd.json_normalize(games_data)

# Resultado
if df_games.empty:
    print("✅ No hay juegos nuevos desde la última actualización.")
else:
    print(f"✅ Total de juegos nuevos: {len(df_games)}")

    # -----------------------------
    # Limpieza y guardado en  formato Delta Lake
    # -----------------------------
    df_games = df_games.fillna("").astype(str)
    almacenamiento_datalake_merge(df_games, path_games, storage_options)


📂 Tabla existente. Revisando última actualización...
📅 Extrayendo juegos actualizados entre 2025-12-07T20:03:33Z y 2025-12-07T20:44:38Z
✅ Total de juegos nuevos: 83
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 0 registros insertados, 83 registros actualizados.


In [10]:
# Me traigo data del repositorio
df_games=DeltaTable(
    path_games,
    storage_options=storage_options
    ).to_pandas()

display(df_games)

,slug,name,playtime,platforms,stores,released,tba,background_image,rating,rating_top,...,added_by_status.yet,added_by_status.owned,added_by_status.beaten,added_by_status.dropped,esrb_rating.id,esrb_rating.name,esrb_rating.slug,esrb_rating.name_en,esrb_rating.name_ru,added_by_status
0,uma-musume-pretty-derby,Uma Musume Pretty Derby,0,"[{'platform': {'id': 21, 'name': 'Android', 's...","[{'store': {'id': 4, 'name': 'App Store', 'slu...",2021-02-24,False,https://media.rawg.io/media/screenshots/70f/70...,0.0,0,...,,,,,,,,,,
1,the-walking-dead-a-new-frontier,The Walking Dead: A New Frontier,9,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2016-12-19,False,https://media.rawg.io/media/games/f0a/f0a65d7d...,3.63,4,...,124.0,1395.0,684.0,46.0,4.0,Mature,mature,Mature,С 17 лет,
2,dying-light-the-beast,Dying Light: The Beast,3,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2025-09-18,False,https://media.rawg.io/media/games/b20/b2055f1b...,3.92,4,...,28.0,22.0,30.0,10.0,,,,,,
3,no-im-not-a-human,No I'm not a Human,8,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2025-09-15,True,https://media.rawg.io/media/games/e85/e852cfb2...,4.03,5,...,12.0,14.0,30.0,3.0,,,,,,
4,potion-craft-alchemist-simulator,Potion Craft: Alchemist Simulator,3,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2022-12-13,False,https://media.rawg.io/media/games/18e/18e422ec...,3.71,4,...,76.0,951.0,76.0,146.0,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,13-sentinels-aegis-rim,13 Sentinels: Aegis Rim,0,"[{'platform': {'id': 187, 'name': 'PlayStation...","[{'store': {'id': 3, 'name': 'PlayStation Stor...",2019-11-28,False,https://media.rawg.io/media/games/be8/be889634...,4.29,5,...,70.0,426.0,118.0,15.0,3.0,Teen,teen,Teen,С 13 лет,
240,hades-ii,Hades II,7,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2025-09-25,False,https://media.rawg.io/media/games/8fd/8fd2e831...,4.39,5,...,85.0,256.0,52.0,24.0,,,,,,
241,ghost-of-tsushima-directors-cut,Ghost of Tsushima Director's Cut,4,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2021-08-20,False,https://media.rawg.io/media/games/193/193c9fe2...,4.59,5,...,77.0,136.0,371.0,45.0,4.0,Mature,mature,Mature,С 17 лет,
242,r4-ridge-racer-type-4-2,R4: Ridge Racer Type 4,0,"[{'platform': {'id': 27, 'name': 'PlayStation'...",,1998-12-03,False,https://media.rawg.io/media/games/04f/04fbf671...,4.04,4,...,3.0,18.0,23.0,11.0,1.0,Everyone,everyone,Everyone,Для всех,


# 4 CONVERSION DE DATOS Y ALMACENAMIENTO EN CAPA SILVER

## 4.1 CARGA DE DATOS

In [11]:
path_base = f"s3://{bkt_name}/bronze/Rawg_Games_API"

# --- Definición de Rutas y Endpoints ---
# Los DataFrames resultantes se guardarán aquí: {nombre_tabla: DataFrame}
df = {}

# 1. Cargar la Tabla 'games' (Nivel Superior)
table_name = "games"
path_bronze = f"{path_base}/{table_name}"

print(f"⏳ Cargando tabla: {table_name} desde {path_bronze}")
try:
    df_games = DeltaTable(
        path_bronze,
        storage_options=storage_options
    ).to_pandas()
    print(f"✅ '{table_name}' cargada. Filas: {len(df_games)}")
except Exception as e:
    print(f"❌ ERROR al cargar {table_name}: {e}")


# 2. Cargar las Tablas de 'metadata' (Sub-Tablas/Endpoints)
metadata_endpoints = ["genres",  "developers", "publishers", "stores", "tags", "creators","platforms"]
metadata_folder = "metadata"

for endpoint in metadata_endpoints:
    table_name = endpoint
    # La ruta esperada: s3://.../Rawg_Games_API/metadata/genres
    path_bronze = f"{path_base}/{metadata_folder}/{endpoint}"
    
    print(f"\n⏳ Cargando tabla: {table_name} desde {path_bronze}")
    try:
        df[table_name] = DeltaTable(
            path_bronze,
            storage_options=storage_options
        ).to_pandas()
        print(f"✅ '{table_name}' cargada. Filas: {len(df[table_name])}")
    except Exception as e:
        print(f"❌ ERROR al cargar {table_name}: {e}")


⏳ Cargando tabla: games desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/games
✅ 'games' cargada. Filas: 244

⏳ Cargando tabla: genres desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/genres
✅ 'genres' cargada. Filas: 19

⏳ Cargando tabla: developers desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/developers
✅ 'developers' cargada. Filas: 40

⏳ Cargando tabla: publishers desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/publishers
✅ 'publishers' cargada. Filas: 40

⏳ Cargando tabla: stores desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/stores
✅ 'stores' cargada. Filas: 10

⏳ Cargando tabla: tags desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/tags
✅ 'tags' cargada. Filas: 40

⏳ Cargando tabla: creators desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadata/creators
✅ 'creators' cargada. Filas: 40

⏳ Cargando tabla: platforms desde s3://pabloluberriaga-bucket/bronze/Rawg_Games_API/metadat

## 4.2 REVISION DE DATOS - GAMES

A los efectos de ordenar la limpieza ordeno mi dataset por temática

In [12]:
# Configuración de Pandas para visualización completa
pd.set_option('display.max_columns', None)

# ---------------------------------------------------------------------------------------------------
# 0. Estandarización inicial de nulos
#  ---------------------------------------------------------------------------------------------------
# Reemplazo '' y strings con solo espacios por np.nan 
df_games = df_games.replace(['', r'^\s*$'], np.nan, regex=True)


# ---------------------------------------------------------------------------------------------------
# 1. DIAGNÓSTICO PREVIO DEL DATAFRAME
# ---------------------------------------------------------------------------------------------------

print("\n" + "---" * 20)
print(" 📊 2.2 Diagnóstico del DataFrame (df_games)")
print("---" * 20)

# Forma (Shape): Filas y Columnas
print(f"📝 Forma del DataFrame (Filas, Columnas): {df_games.shape}")
print(f"Total de Celdas:  {df_games.size:,}")

# Uso Detallado de Memoria (Info)
print("\n" + "---"*10)
print(f"💾 Uso Detallado de Memoria:")
print("\n" + "---"*10)
# Muestra el uso de memoria real, esencial para optimizar la Capa Silver
df_games.info(verbose=False, memory_usage='deep')



# Veo las primeras filas de mi dataframe
print("\n" + "---"*10)
print("  Impresion de mi dataframe")
print("---"*10)
display(df_games.head(10)) 

# ---------------------------------------------------------------------------------------------------
# 1.2 CONTEO DETALLADO DE NULOS
# ---------------------------------------------------------------------------------------------------

# Conteo de nulos después de la estandarización
null_counts = df_games.isnull().sum()
null_cols_detailed = null_counts[null_counts > 0].sort_values(ascending=False)

print("\n" + "---"*30)
print("🛑 Columnas con Valores Nulos (Después de la Limpieza Inicial):")
print("---"*30)

if null_cols_detailed.empty:
    print(" ¡No se encontraron valores nulos en ninguna columna!")
else:
    # Calcular el porcentaje de nulos
    null_percent = (null_cols_detailed / len(df_games)) * 100
    
    # Crear un DataFrame de resumen para una presentación clara
    null_summary = pd.DataFrame({
        'Cantidad Nulos': null_cols_detailed,
        'Porcentaje Nulos': null_percent.map('{:.2f}%'.format) # Formato a dos decimales
    })
    display(null_summary)
    
print("="*50)


------------------------------------------------------------
 📊 2.2 Diagnóstico del DataFrame (df_games)
------------------------------------------------------------
📝 Forma del DataFrame (Filas, Columnas): (244, 42)
Total de Celdas:  10,248

------------------------------
💾 Uso Detallado de Memoria:

------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Columns: 42 entries, slug to added_by_status
dtypes: float64(5), object(37)
memory usage: 3.0 MB

------------------------------
  Impresion de mi dataframe
------------------------------


C:\Users\PC\AppData\Local\Temp\ipykernel_18808\4159528955.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_games = df_games.replace(['', r'^\s*$'], np.nan, regex=True)


,slug,name,playtime,platforms,stores,released,tba,background_image,rating,rating_top,ratings,ratings_count,reviews_text_count,added,metacritic,suggestions_count,updated,id,score,clip,tags,esrb_rating,user_game,reviews_count,community_rating,saturated_color,dominant_color,short_screenshots,parent_platforms,genres,added_by_status.toplay,added_by_status.playing,added_by_status.yet,added_by_status.owned,added_by_status.beaten,added_by_status.dropped,esrb_rating.id,esrb_rating.name,esrb_rating.slug,esrb_rating.name_en,esrb_rating.name_ru,added_by_status
0,uma-musume-pretty-derby,Uma Musume Pretty Derby,0,"[{'platform': {'id': 21, 'name': 'Android', 's...","[{'store': {'id': 4, 'name': 'App Store', 'slu...",2021-02-24,False,https://media.rawg.io/media/screenshots/70f/70...,0.0,0,"[{'id': 5, 'title': 'exceptional', 'count': 2,...",2,0,4,NaN,246,2025-12-07T20:02:33,739667,NaN,NaN,[],NaN,NaN,2,0.0,0f0f0f,0f0f0f,"[{'id': -1, 'image': 'https://media.rawg.io/me...","[{'platform': {'id': 8, 'name': 'Android', 'sl...","[{'id': 14, 'name': 'Simulation', 'slug': 'sim...",1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,the-walking-dead-a-new-frontier,The Walking Dead: A New Frontier,9,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2016-12-19,False,https://media.rawg.io/media/games/f0a/f0a65d7d...,3.63,4,"[{'id': 4, 'title': 'recommended', 'count': 33...",627,3,2351,71.0,622,2025-12-07T19:29:18,9066,NaN,NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,NaN,634,NaN,0f0f0f,0f0f0f,"[{'id': -1, 'image': 'https://media.rawg.io/me...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 3, 'name': 'Adventure', 'slug': 'adven...",89.0,13.0,124.0,1395.0,684.0,46.0,4.0,Mature,mature,Mature,С 17 лет,NaN
2,dying-light-the-beast,Dying Light: The Beast,3,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2025-09-18,False,https://media.rawg.io/media/games/b20/b2055f1b...,3.92,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",20,2,154,NaN,189,2025-12-07T18:17:13,987516,NaN,NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,NaN,24,NaN,0f0f0f,0f0f0f,"[{'id': -1, 'image': 'https://media.rawg.io/me...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...",60.0,4.0,28.0,22.0,30.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN
3,no-im-not-a-human,No I'm not a Human,8,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2025-09-15,True,https://media.rawg.io/media/games/e85/e852cfb2...,4.03,5,"[{'id': 5, 'title': 'exceptional', 'count': 14...",33,0,72,NaN,0,2025-12-07T18:01:53,993426,NaN,NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,NaN,33,NaN,0f0f0f,0f0f0f,"[{'id': -1, 'image': 'https://media.rawg.io/me...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 51, 'name': 'Indie', 'slug': 'indie'},...",13.0,NaN,12.0,14.0,30.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,potion-craft-alchemist-simulator,Potion Craft: Alchemist Simulator,3,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2022-12-13,False,https://media.rawg.io/media/games/18e/18e422ec...,3.71,4,"[{'id': 4, 'title': 'recommended', 'count': 11...",187,0,1377,NaN,150,2025-12-07T18:00:27,554705,NaN,NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,NaN,190,NaN,0f0f0f,0f0f0f,"[{'id': -1, 'image': 'https://media.rawg.io/me...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 40, 'name': 'Casual', 'slug': 'casual'...",92.0,36.0,76.0,951.0,76.0,146.0,NaN,NaN,NaN,NaN,NaN,NaN
5,batman-arkham-city-2,Batman: Arkham City,20,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2011-10-18,False,https://media.rawg.io/media/games/b5a/b5a1226b...,4.41,5,"[{'id': 5, 'title': 'exceptional', 'count': 11...",2154,15,6684,94.0,633,2025-12-07T17:01:11,28623,NaN,NaN,"[{'id':


------------------------------------------------------------------------------------------
🛑 Columnas con Valores Nulos (Después de la Limpieza Inicial):
------------------------------------------------------------------------------------------


,Cantidad Nulos,Porcentaje Nulos
added_by_status,244,100.00%
score,244,100.00%
clip,244,100.00%
esrb_rating,244,100.00%
user_game,244,100.00%
community_rating,164,67.21%
metacritic,149,61.07%
esrb_rating.id,135,55.33%
esrb_rating.name_ru,135,55.33%
esrb_rating.name_en,135,55.33%


## 4.3 PROCESAMIENTO DE DATOS  - GAMES

In [13]:
# ================================================================
# 2.1 NORMALIZACIÓN DE NOMBRES DE COLUMNAS
# ================================================================

df_games.columns = (
    df_games.columns
        .str.lower()
        .str.strip()
        .str.replace(r'[^a-z0-9]+', '_', regex=True)
        .str.replace(r'__+', '_', regex=True)
        .str.strip('_')
)

# ================================================================
# 2.2 ELIMINACIÓN DE COLUMNAS IRRELEVANTES
# ================================================================

columnas_a_eliminar = [
    'rating_top',              # Siempre es 5/100 según el sistema → no aporta variabilidad ni insight.
    'slug',                    # Versión normalizada del nombre → redundante con 'id' y 'name'.
    'esrb_rating_slug',        # Igual al nombre ESRB pero en formato URL → duplicado.
    'dominant_color',          # Metadato visual del juego → no aporta valor a análisis o modelos.
    'saturated_color',         # Igual que dominant_color → metadato asociado a imágenes.
    'short_screenshots',       # Contenido multimedia → peso innecesario y sin utilidad silver.
    'background_image',        # irrelevante para análisis.
    'clip',                    # inutilizable en analítica.
    'reviews_text_count',      # Mayormente nulo .
    'score',                   # 100% nulo
    'user_game',               # 100% nulo
    'esrb_rating',
    'community_rating',        # mayormente vacío.
    'esrb_rating_id',          # Identificador interno del sistema
    'esrb_rating_name_en',     # Duplicado en inglés → no relevante si se trabaja en idioma estándar.
    'esrb_rating_name_ru',     # Igual que el anterior, pero en ruso
]

df_games.drop(columns=[c for c in columnas_a_eliminar if c in df_games.columns], inplace=True)

# ================================================================
# 2.3 RENOMBRADO DE COLUMNAS
# ================================================================

df_games.rename(columns={
    'added': 'added_to_list',
    'released': 'released_date',
    'updated': 'last_updated',
    'playtime': 'average_playtime_hours'
}, inplace=True)

# ================================================================
# 2.4 REORDENAMIENTO DE COLUMNAS
# ================================================================

orden_cols = [
    'id', 'name', 'released_date', 'tba', 'last_updated',
    'genres','platforms', 'parent_platforms', 'stores', 'tags', 'ratings',
    'rating', 'ratings_count', 'metacritic',
    'average_playtime_hours', 'reviews_count', 'suggestions_count',
    'added_to_list',
    'added_by_status_yet', 'added_by_status_owned', 'added_by_status_beaten',
    'added_by_status_toplay', 'added_by_status_dropped', 'added_by_status_playing',
    'esrb_rating_name'
]

df_games = df_games[[c for c in orden_cols if c in df_games.columns]]

# ================================================================
# 3 LIMPIEZA Y TRATAMIENTO DE NULOS
# ================================================================

# 3.2 Drop de registros inválidos
df_games.dropna(subset=['id', 'name'], inplace=True)


# 3.2 Imputación general
imputacion_default = {
    "average_playtime_hours": -1,
    "metacritic": -1,
    "released_date": pd.NaT,
    "last_updated": pd.NaT,
    "added_by_status_yet": 0,
    "added_by_status_owned": 0,
    "added_by_status_beaten": 0,
    "added_by_status_toplay": 0,
    "added_by_status_dropped": 0,
    "added_by_status_playing": 0
}

df_games.fillna(imputacion_default, inplace=True)

# ================================================================
# 4 CASTEO DE TIPOS
# ================================================================

# 4.1 Boolean
df_games['tba'] = (
    df_games['tba']
        .replace(['True', 'true', '1', 1], True)
        .replace(['False', 'false', '0', 0, '', None, 'None', 'nan'], False)
        .astype('boolean')
)

# 4.2 Numéricas
cols_numeric = {
    'id': 'Int64',
    'average_playtime_hours': 'Int64',
    'rating': 'float32',
    'ratings_count': 'Int64',
    'metacritic': 'float32',
    'added_to_list': 'Int64',
    'suggestions_count': 'Int64',
    'reviews_count': 'Int64',
    'added_by_status_yet': 'Int64',
    'added_by_status_owned': 'Int64',
    'added_by_status_beaten': 'Int64',
    'added_by_status_toplay': 'Int64',
    'added_by_status_dropped': 'Int64',
    'added_by_status_playing': 'Int64',
}

for col, tipo in cols_numeric.items():
    if col in df_games.columns:
        df_games[col] = pd.to_numeric(df_games[col], errors='coerce').astype(tipo)

# 4.3 Categorías y strings
df_games["name"] = (
    df_games["name"]
    .replace([None, "", "None"], pd.NA)
    .astype("string")
)

# 2. ESRB como category y imputación de nulos
df_games["esrb_rating_name"] = (
    df_games["esrb_rating_name"]
    .replace([None, "", "None"], "Not rated")  # imputación de nulos
    .astype("category")
)
#Ajusto nombres
df_games['name'] = df_games['name'].str.strip().str.lower().str.title()

# 4.4 Fechas
#Casteo Fechas
df_games['released_date'] = pd.to_datetime(df_games['released_date'], errors='coerce')
df_games['last_updated'] = pd.to_datetime(df_games['last_updated'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')

# Imputación
df_games['released_date'] = df_games['released_date'].fillna(pd.Timestamp('1900-01-01'))
df_games['last_updated'] = df_games['last_updated'].fillna(pd.Timestamp('1900-01-01'))

# Columnas derivadas
df_games['released_year'] = df_games['released_date'].dt.year

# ================================================================
# 5 PROCESAMIENTO DE COLUMNAS ANIDADAS (JSON)
# ================================================================

columnas = ['genres', 'platforms', 'parent_platforms', 'stores', 'tags',"ratings"]
clave = 'id'

# 1. Casteo columnas JSON → listas/dicts válidos
for col in columnas:
    df_games = castear_json(df_games, col)

# 2. Generar dimensiones SIN tablas puente
dimensiones = {}

for col in columnas:
    # saco el contenido completo de la columna
    temp = df_games[col].dropna().explode().dropna()

    # expandir diccionarios a columnas
    temp = temp.apply(pd.Series)

    # Caso especial: plataformas y parent_platforms
    if col in ['platforms', 'parent_platforms']:
        temp = pd.json_normalize(temp['platform'])

    # Caso especial: stores
    if col == 'stores':
        temp = pd.json_normalize(temp['store'])

    # eliminar duplicados
    temp = temp.drop_duplicates(subset='id')

    dimensiones[col] = temp.reset_index(drop=True)

# 3. Limpiar la fact table: dejar solo listas de IDs
for col in columnas:
    df_games = limpiar_json(df_games, col, clave)

for col in columnas:
    df_games[col] = df_games[col].apply(lambda x: json.dumps(x) if isinstance(x, list) else "[]")

C:\Users\PC\AppData\Local\Temp\ipykernel_18808\946254884.py:99: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(['False', 'false', '0', 0, '', None, 'None', 'nan'], False)


## 4.4 PROCESAMIENTO METADATA

In [14]:
# PROCESO DE DIMENSIONES
genres = procesar_dimension(df['genres'], "genre")
developers = procesar_dimension(df['developers'], "developer")
publishers = procesar_dimension(df['publishers'], "publisher")
stores = procesar_dimension(df['stores'], "store", extra_cols=["domain"])
tags = procesar_dimension(df['tags'], "tags")
platforms = procesar_dimension(df['platforms'], "platforms")
parent_platforms = dimensiones['parent_platforms'][['id', 'name']]
creators = procesar_dimension(df['creators'], "creators",extra_cols=["positions","image"])
ratings=dimensiones["ratings"]


# LIMPIEZA ESPECIAL PARA DIM_CREATOR_POSITIONS
# Armo mi tabla de dim_creator_positions

try:
    creators['positions'] = creators['positions'].apply(ast.literal_eval)
except ValueError:
    pass

# 1. Explode de positions
dim_creator_exploded = creators.explode('positions')

# 2. Expando el diccionario en columnas
dim_creator_positions = dim_creator_exploded['positions'].apply(pd.Series)

# 3. Casteo los tipos de datos
dim_creator_positions = dim_creator_positions.astype({
    'id': 'Int64',
    'name': 'string'
})

# 4. Eliminar duplicados por ID
dim_creator_positions = dim_creator_positions.drop_duplicates(subset='id')

# 5. Seleccionar solo columnas deseadas
creator_positions = dim_creator_positions[['id', 'name']]

In [15]:
# Géneros
print("Revision genres")
genres = agregar_faltantes(genres, dimensiones['genres'], id_col='id', name_col='name', count_col='games_count')

# Plataformas
print("Revision plataforms")
platforms = agregar_faltantes(platforms,dimensiones['platforms'],id_col='id',name_col='name',count_col='games_count')

# Stores
print("Revision stores")

stores = agregar_faltantes(stores, dimensiones['stores'], id_col='id', name_col='name', count_col='games_count',extra_cols=['domain'])

# tags
print("Revision tags")
tags=agregar_faltantes(stores, dimensiones['tags'], id_col='id', name_col='name', count_col='games_count')




Revision genres
⚠️ 0 registros faltantes identificados.
Revision plataforms
⚠️ 3 registros faltantes identificados.
Revision stores
⚠️ 0 registros faltantes identificados.
Revision tags
⚠️ 863 registros faltantes identificados.


c:\Users\PC\Desktop\Ingenieria de Datos\Entrega_1\utils.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_origen = pd.concat([df_origen, faltantes], ignore_index=True)
c:\Users\PC\Desktop\Ingenieria de Datos\Entrega_1\utils.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_origen = pd.concat([df_origen, faltantes], ignore_index=True)


## 4.5 ALMACENAMIENTO EN CAPA SILVER

In [16]:
# ---------------------------------------------------------
# RUTAS SILVER
# ---------------------------------------------------------
path_silver_root = f"s3://{bkt_name}/silver/Rawg_Games_API/"
path_games = f"{path_silver_root}games/"
path_metadata = f"{path_silver_root}metadata/"

# ---------------------------------------------------------
# 1. FACT TABLE: GAMES
# ---------------------------------------------------------
almacenamiento_datalake_merge(df_games, path_games, storage_options,partition_by=['released_year'])

# ---------------------------------------------------------
# 2. DIMENSIONES SILVER
# ---------------------------------------------------------
dimensiones = {
    "genres": genres,
    "developers": developers,
    "publishers": publishers,
    "stores": stores,
    "tags": tags,
    "platforms": platforms,
    "creators": creators,
    "creator_positions": creator_positions,
    "parent_platforms": parent_platforms,
    "ratings": ratings
}

for nombre, df_dim in dimensiones.items():
    ruta = f"{path_metadata}{nombre}/"
    almacenamiento_datalake_merge(df_dim, ruta, storage_options)


📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 244 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 19 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 40 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 40 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 10 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 873 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correctamente. 43 registros insertados, 0 registros actualizados.
📂 La tabla Delta ya existe. Ejecutando MERGE...
✅ Merge ejecutado correcta

# 5. CONVERSIÓN DE DATOS Y ALMACENAMIETNO EN CAPA GOLD

## 5.1 CARGA DE DATOS CAPA SILVER

In [17]:
# Definición de rutas base 
path_base = f"s3://{bkt_name}/silver/Rawg_Games_API"


# --- Diccionario para guardar los DataFrames ---
df = {}

# 1. Cargar la tabla 'games'
table_name = "games"
path_silver_games = f"{path_base}/{table_name}"

print(f"⏳ Cargando tabla: {table_name} desde {path_silver_games}")
try:
    df_games = DeltaTable(
        path_silver_games,
        storage_options=storage_options
    ).to_pandas()
    print(f"✅ '{table_name}' cargada. Filas: {len(df_games)}")
except Exception as e:
    print(f"❌ ERROR al cargar {table_name}: {e}")

# 2. Cargar las tablas de metadata
metadata_endpoints = [
    "genres", 
    "developers",
    "publishers",
    "stores",
    "tags",
    "platforms",
    "creators",
    "creator_positions",
    "parent_platforms",
    "ratings"
]

metadata_folder = "metadata"

for endpoint in metadata_endpoints:
    table_name = endpoint
    path_silver_meta = f"{path_base}/{metadata_folder}/{endpoint}"
    
    print(f"\n⏳ Cargando tabla: {table_name} desde {path_silver_meta}")
    try:
        df[table_name] = DeltaTable(
            path_silver_meta,
            storage_options=storage_options
        ).to_pandas()
        print(f"✅ '{table_name}' cargada. Filas: {len(df[table_name])}")
    except Exception as e:
        print(f"❌ ERROR al cargar {table_name}: {e}")


⏳ Cargando tabla: games desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/games
✅ 'games' cargada. Filas: 244

⏳ Cargando tabla: genres desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/metadata/genres
✅ 'genres' cargada. Filas: 19

⏳ Cargando tabla: developers desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/metadata/developers
✅ 'developers' cargada. Filas: 40

⏳ Cargando tabla: publishers desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/metadata/publishers
✅ 'publishers' cargada. Filas: 40

⏳ Cargando tabla: stores desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/metadata/stores
✅ 'stores' cargada. Filas: 10

⏳ Cargando tabla: tags desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/metadata/tags
✅ 'tags' cargada. Filas: 873

⏳ Cargando tabla: platforms desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/metadata/platforms
✅ 'platforms' cargada. Filas: 43

⏳ Cargando tabla: creators desde s3://pabloluberriaga-bucket/silver/Rawg_Games_API/meta

## 5.2  TRANSFORMACION DE DATOS CAPA GOLD

| Dataset GOLD            | Pregunta de Negocio que responde                                                                                               | Descripción                                                                                                                |
| ----------------------- | ------------------------------------------------------------------------------------------------------------------------------ | -------------------------------------------------------------------------------------------------------------------------- |
| **df_kpis_games**       | **¿Cuál es el estado general del catálogo de juegos? ¿Cómo rinden en promedio en términos de rating, engagement y actividad?** | Tabla de KPIs generales: conteo total, ratings medios, playtime, engagement, tasas de completado y abandono.               |
| **df_top_rated**        | **¿Cuáles son los mejores juegos según la valoración de los usuarios?**                                                        | Lista de los juegos con mejor rating filtrando mínimo de participación.                                                    |
| **df_top_rated_yearly** | **¿Qué juegos destacaron año a año según el rating?**                                                                          | Top 10 juegos por cada año ordenados por rating.                                                                           |
| **df_most_played**      | **¿Qué juegos generan mayor engagement y volumen de usuarios?**                                                                | Ranking por `added_to_list` (juegos más jugados / populares).                                                              |
| **df_top_metacritic**   | **¿Qué juegos tienen mejor recepción crítica profesional?**                                                                    | Ranking de los mejores juegos por Metacritic > 0.                                                                          |
| **df_yearly_trends**    | **¿Cómo evolucionan las tendencias del mercado gaming a lo largo del tiempo?**                                                 | Evolución por año: cantidad de lanzamientos, rating medio, engagement, horas promedio.                                     |
| **df_engagement**       | **¿Qué juegos tienen mayor tasa de completado, abandono y actividad?**                                                         | Tabla GOLD de engagement: completion, dropout y active playing rates.                                                      |
| **df_platform_stats**   | **¿Qué plataformas muestran mejor rendimiento en ratings, engagement y cantidad de juegos?**                                   | Agregaciones por plataforma: cantidad de juegos, rating promedio, favoritos, etc.                                          |
| **df_segment_summary**  | **¿Cómo se comportan grupos de juegos según género, plataforma, año o segmentos definidos?**                                   | Resumen segmentado (dependiendo cómo lo construiste): promedios, medianas, conteos y métricas de performance por clusters. |


#### 5.2.1 KPIS GENERALES

In [18]:
print("\n================================================")
print(" KPIs Generales de Juegos")
print("================================================\n")

df_status_stats = df_games.copy()

# Trato divisones por 0
added_to_list_clean=df_games['added_to_list'].replace(0, np.nan)

# -------------------------
# Cálculo de completados y abandonados
# -------------------------
df_status_stats['completion_rate'] = ((df_games['added_by_status_beaten'] / added_to_list_clean) * 100).astype('float').round(2)

df_status_stats['dropout_rate'] = ((df_games['added_by_status_dropped'] / added_to_list_clean) * 100).astype('float').round(2)

# -------------------------
# Tabla de KPIs
# -------------------------
df_kpis_games = pd.DataFrame({
    'metric': [
        'Total Games',
        'Average Rating',
        'Average Metacritic Score',
        'Total User Engagement (added_to_list)',
        'Average Playtime (hours)',
        'Total Reviews',
        'Games with >1000 users',
        'Games with rating >4.0',
        'Average Completion Rate (%)',
        'Average Dropout Rate (%)'
    ],
    'value': [
        len(df_games),
        df_games['rating'].mean().round(2),
        df_games[df_games['metacritic'] > 0]['metacritic'].mean().round(2),
        df_games['added_to_list'].sum(),
        df_games[df_games['average_playtime_hours'] > 0]['average_playtime_hours'].mean().round(2),
        df_games['reviews_count'].sum(),
        df_games[df_games['added_to_list'] > 1000].shape[0],
        df_games[df_games['rating'] > 4.0].shape[0],
        df_status_stats['completion_rate'].mean(skipna=True).round(2),
        df_status_stats['dropout_rate'].mean(skipna=True).round(2)
    ]
})
print("Muestra de métricas clave:")
display(df_kpis_games)



 KPIs Generales de Juegos

Muestra de métricas clave:


,metric,value
0,Total Games,244.000000
1,Average Rating,2.520000
2,Average Metacritic Score,80.349998
3,Total User Engagement (added_to_list),512220.000000
4,Average Playtime (hours),10.310000
5,Total Reviews,116620.000000
6,Games with >1000 users,91.000000
7,Games with rating >4.0,86.000000
8,Average Completion Rate (%),14.900000
9,Average Dropout Rate (%),24.000000


#### 5.2.2 Rankings y Tops

In [19]:
print("\n================================================")
print(" Rankings y Tops")
print("================================================\n")

# ------------------------------------------
# Top 10 juegos mejor valorados (mín 50 puntajes)
# ------------------------------------------
df_games_filtered=(df_games[df_games['ratings_count'] >= 50])
df_top_rated = (
    df_games_filtered[df_games_filtered['ratings_count'] >= 50]
    .nlargest(10, 'rating')[['name', 'rating', 'ratings_count', 'released_year']]
    .reset_index(drop=True))

print(" Top 10 juegos mejor valorados:")
display(df_top_rated)

# ------------------------------------------
# Top 10 por año
# ------------------------------------------
df_top_rated_yearly = (
    df_games_filtered
    .sort_values(['released_year', 'rating'], ascending=[True, False])
    .groupby('released_year')
    .head(10)[['released_year', 'name', 'rating', 'ratings_count']]
    .reset_index(drop=True)
)
print("Top juegos por año (mín 50 ratings):")
display(df_top_rated_yearly)

# ------------------------------------------
# Más jugados
# ------------------------------------------
df_most_played = (
    df_games
    .nlargest(10, 'added_to_list')[['name', 'added_to_list', 'average_playtime_hours', 'rating']]
    .reset_index(drop=True)
)

print("Top 10 juegos más jugados (según added_to_list):")
display(df_most_played)

# ------------------------------------------
# Top por Metacritic
# ------------------------------------------
df_top_metacritic = (
    df_games[df_games['metacritic'] > 0] # Descarto metacritic -1
    .nlargest(10, 'metacritic')[['name', 'metacritic', 'rating', 'released_year']]
    .reset_index(drop=True)
)

print("Top 10 Metacritic :")
display(df_top_metacritic)





 Rankings y Tops

 Top 10 juegos mejor valorados:


,name,rating,ratings_count,released_year
0,Dispatch,4.76,70,2025
1,The Last Of Us Part I,4.67,528,2022
2,Resident Evil 4,4.60,565,2023
3,Red Dead Redemption 2,4.59,5404,2018
4,Ghost Of Tsushima Director'S Cut,4.59,305,2021
5,Clair Obscur: Expedition 33,4.53,312,2025
6,The Last Of Us: Left Behind,4.46,571,2014
7,It Takes Two,4.46,1675,2021
8,Grand Theft Auto: Vice City,4.44,3062,2002
9,Minecraft,4.43,3075,2009


Top juegos por año (mín 50 ratings):


,released_year,name,rating,ratings_count
0,1991,The Legend Of Zelda: A Link To The Past,4.35,578
1,1996,"Pokémon Red, Blue, Yellow",4.39,532
2,1998,The Legend Of Zelda: Ocarina Of Time,4.38,869
3,2000,Spider-Man (2000),4.29,396
4,2000,The Operative: No One Lives Forever,4.22,125
...,...,...,...,...
109,2025,Hades Ii,4.39,86
110,2025,Kingdom Come: Deliverance Ii,4.36,113
111,2025,The Alters,4.35,60
112,2025,Hollow Knight: Silksong,4.33,186


Top 10 juegos más jugados (según added_to_list):


,name,added_to_list,average_playtime_hours,rating
0,The Elder Scrolls V: Skyrim,16727,46,4.42
1,Red Dead Redemption 2,16668,21,4.59
2,Cyberpunk 2077,13806,29,4.22
3,Grand Theft Auto Iv,13114,10,4.26
4,Hollow Knight,11755,7,4.40
5,Middle-Earth: Shadow Of Mordor,11716,13,3.91
6,Dark Souls Iii,11166,29,4.40
7,Stardew Valley,10928,13,4.39
8,Far Cry 3,10728,16,4.25
9,Fallout: New Vegas,10169,12,4.42


Top 10 Metacritic :


,name,metacritic,rating,released_year
0,The Legend Of Zelda: Ocarina Of Time,99.0,4.38,1998
1,Red Dead Redemption 2,96.0,4.59,2018
2,The Legend Of Zelda: Tears Of The Kingdom,96.0,4.38,2023
3,Elden Ring,95.0,4.39,2022
4,Red Dead Redemption,95.0,4.42,2010
5,Grand Theft Auto Iv,95.0,4.26,2008
6,The Legend Of Zelda: A Link To The Past,95.0,4.35,1991
7,Batman: Arkham City,94.0,4.41,2011
8,The Elder Scrolls V: Skyrim,94.0,4.42,2011
9,Metal Gear Solid 4: Guns Of The Patriots,94.0,4.38,2008


#### 5.2.3 Tendencias anuales

In [20]:
print("\n================================================")
print(" Tedencias desde el 2000")
print("================================================\n")

df_yearly_trends = (
    df_games[df_games['released_year'] >= 2000]
    .groupby('released_year')
    .agg({
        'id': 'count',
        'rating': 'mean',
        'metacritic': lambda x: x[x > 0].mean() if (x > 0).any() else None,
        'added_to_list': 'sum',
        'average_playtime_hours': lambda x: x[x > 0].mean()
    })
    .rename(columns={
        'id': 'total_games',
        'rating': 'avg_rating',
        'metacritic': 'avg_metacritic',
        'added_to_list': 'total_users',
        'average_playtime_hours': 'avg_playtime'
    })
    .round(2)
    .reset_index()
)

print("📈 Tendencias anuales :")
display(df_yearly_trends)




 Tedencias desde el 2000

📈 Tendencias anuales :


,released_year,total_games,avg_rating,avg_metacritic,total_users,avg_playtime
0,2000,3,3.98,70.67,1173,NaN
1,2001,2,4.25,87.00,10977,1.00
2,2002,1,4.44,94.00,9812,2.00
3,2004,4,3.77,76.25,4969,19.50
4,2005,2,2.20,83.00,7012,3.00
5,2006,3,3.74,63.67,6042,2.00
6,2007,2,3.00,57.50,278,1.00
7,2008,2,4.32,94.50,15292,15.00
8,2009,4,4.20,84.75,21969,11.25
9,2010,6,3.58,81.75,29177,5.50


#### 5.2.4 User Engagement 

In [21]:
print("\n================================================")
print("           Tabla de Engagement                    ")
print("================================================\n")

# ------------------------------------------
# Cálculo de active_playing_rate
# ------------------------------------------
df_status_stats['active_playing_rate'] = (
    df_status_stats['added_by_status_playing'] / added_to_list_clean * 100
).round(1)

# ------------------------------------------
# Construcción de la tabla GOLD de Engagement
# ------------------------------------------
df_engagement = (
    df_status_stats[df_status_stats['added_to_list'] >= 100]
    [['name', 'added_to_list', 'completion_rate', 'dropout_rate',
      'active_playing_rate', 'rating']]
    .sort_values('completion_rate', ascending=False)
    .head(20)
    .reset_index(drop=True)
)

print("Top 20 juegos con mayor tasa de completado:\n")
display(df_engagement)


           Tabla de Engagement                    

Top 20 juegos con mayor tasa de completado:



,name,added_to_list,completion_rate,dropout_rate,active_playing_rate,rating
0,The Last Of Us: Left Behind,975,75.69,1.64,0.9,4.46
1,Monument Valley,1057,63.67,8.04,1.2,4.40
2,"Pokémon Red, Blue, Yellow",1098,63.21,8.74,1.6,4.39
3,Spider-Man (2000),804,56.22,15.42,0.6,4.29
4,Metal Gear Solid 3: Snake Eater,1700,45.88,4.29,1.4,4.36
5,Assassin'S Creed Ii,8123,44.44,4.85,0.6,4.42
6,The Legend Of Zelda: Ocarina Of Time,1861,42.88,9.73,2.8,4.38
7,Rayman Raving Rabbids 2,105,41.90,17.14,0.0,3.57
8,Dispatch,219,41.10,1.83,10.0,4.76
9,Ghost Of Tsushima Director'S Cut,920,40.33,4.89,6.6,4.59


#### 5.2.4 Análisis por plataforma

In [22]:
# Expandir plataformas
df_platform_exploded = df_games.copy()
df_platform_exploded['platforms'] = df_platform_exploded['platforms'].apply(json.loads)
df_platform_exploded = df_platform_exploded.explode('platforms')

# Merge con dimensión
df_platform_exploded['platform_id'] = df_platform_exploded['platforms'].astype('Int64')
df_games_with_platforms = df_platform_exploded.merge(
    df['platforms'][['id', 'name']], 
    left_on='platform_id', 
    right_on='id', 
    suffixes=('', '_platform')
)

# Agregación
df_platform_stats = (
    df_games_with_platforms
    .groupby('name_platform')
    .agg({
        'id': 'count',
        'rating': 'mean',
        'added_to_list': 'sum',
        'average_playtime_hours': lambda x: x[x > 0].mean()
    })
    .rename(columns={
        'id': 'total_games',
        'rating': 'avg_rating',
        'added_to_list': 'total_users',
        'average_playtime_hours': 'avg_playtime'
    })
    .round(2)
    .sort_values('total_games', ascending=False)
    .reset_index()
)

df_platform_stats

,name_platform,total_games,avg_rating,total_users,avg_playtime
0,Pc,204,2.73,499695,10.04
1,Playstation 4,77,3.89,392165,11.01
2,Xbox One,64,3.94,366523,11.59
3,Playstation 5,52,3.44,98710,13.48
4,Nintendo Switch,51,3.62,185773,11.46
5,Macos,50,3.48,203431,8.07
6,Xbox Series S/X,45,3.30,90464,13.03
7,Linux,35,3.01,131755,13.41
8,Playstation 3,29,3.94,178697,10.92
9,Ios,28,3.75,109103,9.04


#### 5.2.5 Análisis por Tags

In [23]:
print("===============================================")
print("  STATS POR TAG")
print("===============================================")

# ------------------------------------------------
# Parseo y Explode de Tags
# ------------------------------------------------

df_games_tags = df_games.copy()
df_games_tags['tags'] = df_games_tags['tags'].apply(json.loads)

df_games_tags = df_games_tags.explode('tags')
df_games_tags['tag_id'] = df_games_tags['tags'].astype('Int64')

# ------------------------------------------------
# Merge con Dimensión TAGS
# ------------------------------------------------

df_games_with_tags = df_games_tags.merge(
    df['tags'][['id', 'name']],
    left_on='tag_id',
    right_on='id',
    suffixes=('', '_tag')
)


# ------------------------------------------------
#  Agregación GOLD por TAG
# ------------------------------------------------

df_tags = (
    df_games_with_tags
    .groupby('name_tag')
    .agg({
        'id': 'count',                                # total de juegos
        'rating': 'mean',                             # rating promedio
        'metacritic': lambda x: x[x > 0].mean(),      # metacritic válido
        'added_to_list': 'sum',                       # usuarios totales
        'reviews_count': 'sum',
        'ratings_count': 'sum',
        'average_playtime_hours': lambda x: x[x > 0].mean()
    })
    .rename(columns={
        'id': 'total_games',
        'rating': 'avg_rating',
        'metacritic': 'avg_metacritic',
        'added_to_list': 'total_users',
        'reviews_count': 'total_reviews',
        'ratings_count': 'total_ratings',
        'average_playtime_hours': 'avg_playtime'
    })
    .round(2)
    .sort_values('total_users', ascending=False)
    .reset_index()
)

display(df_tags.head(20))


  STATS POR TAG


,name_tag,total_games,avg_rating,avg_metacritic,total_users,total_reviews,total_ratings,avg_playtime
0,Singleplayer,159,3.03,81.11,462300,100766,99290,10.31
1,Для одного игрока,130,3.28,81.38,443496,95613,94228,9.51
2,Экшен,104,3.36,80.89,387185,83826,82577,10.19
3,Приключение,105,3.33,80.35,379946,84168,82945,8.96
4,Atmospheric,86,3.57,82.40,376167,86703,85471,10.36
5,Атмосфера,53,3.89,82.74,346968,79349,78287,10.02
6,Открытый мир,55,3.85,81.44,301885,67362,66371,11.44
7,Open World,54,3.87,81.98,299648,67415,66409,11.48
8,Глубокий сюжет,75,3.52,82.30,281399,66304,65201,11.31
9,Story Rich,70,3.44,82.69,271818,64686,63616,11.73


In [24]:

print("===============================================")
print("  STAT POR SEGMENTO ")
print("===============================================")
df_segment_summary = df_games.copy()

# Definir segmentos
df_segment_summary['popularity_tier'] = pd.cut(
    df_segment_summary['added_to_list'],
    bins=[0, 100, 1000, 10000, float('inf')],
    labels=['Niche', 'Indie Popular', 'Mainstream', 'Blockbuster']
)

# Resumen por segmento
df_segment_summary = (
    df_segment_summary
    .groupby('popularity_tier', observed=True)
    .agg({
        'id': 'count',
        'rating': 'mean',
        'average_playtime_hours': lambda x: x[x > 0].mean()
    })
    .rename(columns={'id': 'game_count', 'rating': 'avg_rating', 'average_playtime_hours': 'avg_playtime'})
    .round(2)
    .reset_index()
)
display(df_segment_summary)


  STAT POR SEGMENTO 


,popularity_tier,game_count,avg_rating,avg_playtime
0,Niche,108,0.77,12.40
1,Indie Popular,44,3.86,8.12
2,Mainstream,81,3.95,9.36
3,Blockbuster,10,4.33,19.60


In [25]:
# 5.3 ALMACENAMIENTO EN CAPA GOLD

In [ ]:
# ============================================================
#  ALMACENAMIENTO CAPA GOLD - RAWG GAMES API
# ============================================================

# Ruta raíz GOLD
path_gold_root = f"s3://{bkt_name}/gold/Rawg_Games_API/"

# Diccionario: { carpeta_gold : dataframe }
gold_tables = {
    "kpis_games": df_kpis_games,
    "top_rated": df_top_rated,
    "top_rated_yearly": df_top_rated_yearly,
    "most_played": df_most_played,
    "top_metacritic": df_top_metacritic,
    "yearly_trends": df_yearly_trends,
    "engagement": df_engagement,
    "platform_stats": df_platform_stats,
    "segment_summary": df_segment_summary
}

# ------------------------------------------------------------
# GUARDADO GOLD
# ------------------------------------------------------------
for table_name, df_table in gold_tables.items():

    path_table = f"{path_gold_root}{table_name}/"

    print(f"📁 Guardando GOLD: {table_name} ...")
   
    almacenamiento_datalake_overwrite(
        df_table,
        path_table,
        storage_options
    )

    print(f" GOLD '{table_name}' sobrescrito correctamente.\n")



📁 Guardando GOLD: kpis_games ...
→ Ruta: s3://pabloluberriaga-bucket/gold/Rawg_Games_API/kpis_games/
✔ GOLD 'kpis_games' sobrescrito correctamente.

📁 Guardando GOLD: top_rated ...
→ Ruta: s3://pabloluberriaga-bucket/gold/Rawg_Games_API/top_rated/
✔ GOLD 'top_rated' sobrescrito correctamente.

📁 Guardando GOLD: top_rated_yearly ...
→ Ruta: s3://pabloluberriaga-bucket/gold/Rawg_Games_API/top_rated_yearly/
✔ GOLD 'top_rated_yearly' sobrescrito correctamente.

📁 Guardando GOLD: most_played ...
→ Ruta: s3://pabloluberriaga-bucket/gold/Rawg_Games_API/most_played/
✔ GOLD 'most_played' sobrescrito correctamente.

📁 Guardando GOLD: top_metacritic ...
→ Ruta: s3://pabloluberriaga-bucket/gold/Rawg_Games_API/top_metacritic/
✔ GOLD 'top_metacritic' sobrescrito correctamente.

📁 Guardando GOLD: yearly_trends ...
→ Ruta: s3://pabloluberriaga-bucket/gold/Rawg_Games_API/yearly_trends/
✔ GOLD 'yearly_trends' sobrescrito correctamente.

📁 Guardando GOLD: engagement ...
→ Ruta: s3://pabloluberriaga-bucke